In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_seq_items', 100)

# 400 meter coordinate accuracy
# 1924 earliest year

In [ ]:
df_path = input("Enter the file path: ")
df = pd.read_parquet(df_path)

In [ ]:
checklist_path = input("Enter the checklist path: ")
checklist = pd.read_parquet(checklist_path)

In [ ]:
df

In [ ]:
checklist

In [ ]:
# reset index on df and checklist
df = df.reset_index(drop=True)
checklist = checklist.reset_index(drop=True)

In [ ]:
duplicates_df = df[df.duplicated(["scientific_name"], keep=False)]
duplicates_df

In [ ]:
# rename canonicalName to scientific_name in checklist
checklist.rename(columns={"canonicalName": "scientific_name"}, inplace=True)

In [ ]:
duplicates_checklist = checklist[checklist.duplicated(["scientific_name"], keep=False)]
duplicates_checklist

In [ ]:
df_in_checklist = df[df["scientific_name"].isin(checklist["scientific_name"])]
df_in_checklist

In [ ]:
df_not_in_checklist = df[~df["scientific_name"].isin(checklist["scientific_name"])]
df_not_in_checklist

In [ ]:
df_in_duplicates_checklist = df[
    df["scientific_name"].isin(duplicates_checklist["scientific_name"])
]
df_in_duplicates_checklist

In [ ]:
df_clean = pd.merge(
    df,
    checklist,
    how="inner",
    on=["kingdom", "phylum", "class", "order", "family", "genus", "scientific_name"],
)
df_clean

In [ ]:
# df_dirty equals rows of df not in df_cleean
df_dirty = df[~df.index.isin(df_clean.index)]
df_dirty

In [ ]:
# drop columns taxonRank and taxonomicStatus
df_clean = df_clean.drop(["taxonRank", "taxonomicStatus"], axis=1)
df_clean

In [ ]:
# fill missing values of native_ecoregions and freshwater_ecoregions with empty list
for i in range(len(df_clean)):
    print(i)
    #    print(type(df_clean.loc[i, "native_ecoregions"]))
    if df_clean.loc[i, "native_ecoregions"] is None:
        df_clean.loc[i, "native_ecoregions"] = [None]
    if df_clean.loc[i, "freshwater_ecoregions"] is None:
        df_clean.loc[i, "freshwater_ecoregions"] = [None]


df_clean

In [ ]:
df_clean = df_clean.groupby(["acceptedName"], as_index=False, dropna=False).apply(
    lambda x: pd.Series(
        {
            "observed_ecoregions": list(
                [item for sublist in x["observed_ecoregions"] for item in sublist]
            ),
            "rights": list([item for sublist in x["rights"] for item in sublist]),
            "native_ecoregions": list(
                set(
                    [
                        item
                        for sublist in x["native_ecoregions"]
                        for item in sublist
                        if item is not None
                    ]
                )
            ),
            "freshwater_ecoregions": list(
                set(
                    [
                        item
                        for sublist in x["freshwater_ecoregions"]
                        for item in sublist
                        if item is not None
                    ]
                )
            ),
            "common_name": list(x["common_name"].unique()),
            "species_type": list(x["species_type"].unique()),
            "scientific_name": list(x["scientific_name"].unique()),
            "kingdom": list(x["kingdom"].unique()),
            "phylum": list(x["phylum"].unique()),
            "class": list(x["class"].unique()),
            "order": list(x["order"].unique()),
            "family": list(x["family"].unique()),
            "genus": list(x["genus"].unique()),
        }
    )
)

In [ ]:
df_clean

In [ ]:
# find rows of df_clean where length of scientific_name is greater than 1
scientific_dups = df_clean[df_clean["scientific_name"].str.len() > 1]

scientific_dups

In [ ]:
kingdom_dups = df_clean[df_clean["kingdom"].str.len() > 1]

kingdom_dups

In [ ]:
phylum_dups = df_clean[df_clean["phylum"].str.len() > 1]

phylum_dups

In [ ]:
class_dups = df_clean[df_clean["class"].str.len() > 1]

class_dups

In [ ]:
order_dups = df_clean[df_clean["order"].str.len() > 1]

order_dups

In [ ]:
family_dups = df_clean[df_clean["family"].str.len() > 1]

family_dups

In [ ]:
genus_dups = df_clean[df_clean["genus"].str.len() > 1]

genus_dups

In [ ]:
df_clean = df_clean[~df_clean.index.isin(kingdom_dups.index)]

df_clean

In [ ]:
df_clean = df_clean.reset_index(drop=True)

df_clean

In [ ]:
species_type_dups = df_clean[df_clean["species_type"].str.len() > 1]

species_type_dups

In [ ]:
for i in range(len(df_clean)):
    print(i)
    df_clean.loc[i, "kingdom"] = df_clean.loc[i, "kingdom"][0]
    df_clean.loc[i, "phylum"] = df_clean.loc[i, "phylum"][0]
    df_clean.loc[i, "class"] = df_clean.loc[i, "class"][0]
    df_clean.loc[i, "order"] = df_clean.loc[i, "order"][0]
    df_clean.loc[i, "family"] = df_clean.loc[i, "family"][0]
    df_clean.loc[i, "genus"] = df_clean.loc[i, "genus"][0]
    df_clean.loc[i, "species_type"] = df_clean.loc[i, "species_type"][0]


df_clean

In [ ]:
common_name_dups = df_clean[df_clean["common_name"].str.len() > 1]

common_name_dups

In [ ]:
df_clean["common_name"] = df_clean["common_name"].apply(
    lambda x: [i for i in x if i is not None]
)

df_clean

In [ ]:
for i in range(len(df_clean)):
    print(i)
    if len(df_clean.loc[i, "common_name"]) == 0:
        df_clean.loc[i, "common_name"] = None
    else:
        df_clean.loc[i, "common_name"] = df_clean.loc[i, "common_name"][0]

df_clean

In [ ]:
df_clean

In [ ]:
# drop column scientific_name
df_clean = df_clean.drop(["scientific_name"], axis=1)

df_clean

In [ ]:
# rename acceptedName to scientific_name
df_clean = df_clean.rename(columns={"acceptedName": "scientific_name"})

df_clean

In [ ]:
print(f'max ecoregions length is {df_clean["observed_ecoregions"].str.len().max()}')
print(f'max rights length is {df_clean["rights"].str.len().max()}')

In [ ]:
for i in range(len(df_clean)):
    print(i)

In [ ]:
# SLOW, TOOK ABOUT AN HOUR FOR 300,000+ ROWS
for i in range(len(df_clean)):
    print(f"index: {i}")
    # print(len(test["observed_ecoregions"].iloc[i]))
    cat = []
    rights_cat = []
    for j in range(len(df_clean["observed_ecoregions"].iloc[i])):
        if df_clean["observed_ecoregions"].iloc[i][j] not in cat:
            cat.append(df_clean["observed_ecoregions"].iloc[i][j])
            rights_cat.append(df_clean["rights"].iloc[i][j])
        # print(test["observed_ecoregions"].iloc[i][j])
        # for k in test["observed_ecoregions"].iloc[i][j]:
        # print(k)
        # if k not in cat:
        #     cat.append(k)
    # if len(cat) != len(test["observed_ecoregions"].iloc[i]):
    # print(f'index: {i}')

    # print(f'observed: {test["observed_ecoregions"].iloc[i]}')
    # print(f'cat: {cat}')
    if len(df_clean["rights"].iloc[i]) != len(df_clean["observed_ecoregions"].iloc[i]):
        print(f"index: {i}")
        print("rights")

    df_clean["observed_ecoregions"].iloc[i] = cat
    df_clean["rights"].iloc[i] = rights_cat

In [ ]:
df_clean

In [ ]:
df_clean.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/occurences/step_4/processing/step_3/checklist/clean_checklisted.parquet"
)

In [ ]:
df_dirty.to_parquet(
    "/media/muskrat/T7 Shield/eco_data/v4/occurences/step_4/processing/step_3/checklist/dirty_checklisted.parquet"
)